In [14]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 18: Many-Worlds Interpretation ---</h2><p>As you approach Neptune, a planetary security system detects you and activates a giant <a href="https://en.wikipedia.org/wiki/Tractor_beam">tractor beam</a> on <a href="https://en.wikipedia.org/wiki/Triton_(moon)">Triton</a>!  You have no choice but to land.</p>
<p>A scan of the local area reveals only one interesting feature: a massive underground vault.  You generate a map of the tunnels (your puzzle input).  The tunnels are too narrow to move diagonally.</p>
<p>Only one <em>entrance</em> (marked <code>@</code>) is present among the <em>open passages</em> (marked <code>.</code>) and <em>stone walls</em> (<code>#</code>), but you also detect an assortment of <em>keys</em> (shown as lowercase letters) and <em>doors</em> (shown as uppercase letters). Keys of a given letter open the door of the same letter: <code>a</code> opens <code>A</code>, <code>b</code> opens <code>B</code>, and so on.  You aren't sure which key you need to disable the tractor beam, so you'll need to <em>collect all of them</em>.</p>
<p>For example, suppose you have the following map:</p>
<pre><code>#########
#b.A.@.a#
#########
</code></pre>
<p>Starting from the entrance (<code>@</code>), you can only access a large door (<code>A</code>) and a key (<code>a</code>). Moving toward the door doesn't help you, but you can move <code>2</code> steps to collect the key, unlocking <code>A</code> in the process:</p>
<pre><code>#########
#b.....@#
#########
</code></pre>
<p>Then, you can move <code>6</code> steps to collect the only other key, <code>b</code>:</p>
<pre><code>#########
#@......#
#########
</code></pre>
<p>So, collecting every key took a total of <code><em>8</em></code> steps.</p>
<p>Here is a larger example:</p>
<pre><code>########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################
</code></pre>
<p>The only reasonable move is to take key <code>a</code> and unlock door <code>A</code>:</p>
<pre><code>########################
#f.D.E.e.C.b.....@.B.c.#
######################.#
#d.....................#
########################
</code></pre>
<p>Then, do the same with key <code>b</code>:</p>
<pre><code>########################
#f.D.E.e.C.@.........c.#
######################.#
#d.....................#
########################
</code></pre>
<p>...and the same with key <code>c</code>:</p>
<pre><code>########################
#f.D.E.e.............@.#
######################.#
#d.....................#
########################
</code></pre>
<p>Now, you have a choice between keys <code>d</code> and <code>e</code>.  While key <code>e</code> is closer, collecting it now would be slower in the long run than collecting key <code>d</code> first, so that's the best choice:</p>
<pre><code>########################
#f...E.e...............#
######################.#
#@.....................#
########################
</code></pre>
<p>Finally, collect key <code>e</code> to unlock door <code>E</code>, then collect key <code>f</code>, taking a grand total of <code><em>86</em></code> steps.</p>
<p>Here are a few more examples:</p>
<ul>
<li><pre><code>########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################
</code></pre>
<p>Shortest path is <code>132</code> steps: <code>b</code>, <code>a</code>, <code>c</code>, <code>d</code>, <code>f</code>, <code>e</code>, <code>g</code></p></li>
<li><pre><code>#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
</code></pre>
<p>Shortest paths are <code>136</code> steps;<br>one is: <code>a</code>, <code>f</code>, <code>b</code>, <code>j</code>, <code>g</code>, <code>n</code>, <code>h</code>, <code>d</code>, <code>l</code>, <code>o</code>, <code>e</code>, <code>p</code>, <code>c</code>, <code>i</code>, <code>k</code>, <code>m</code></p></li>
<li><pre><code>########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################
</code></pre>
<p>Shortest paths are <code>81</code> steps; one is: <code>a</code>, <code>c</code>, <code>f</code>, <code>i</code>, <code>d</code>, <code>g</code>, <code>b</code>, <code>e</code>, <code>h</code></p></li>
</ul>
<p><em>How many steps is the shortest path that collects all of the keys?</em></p>
</article>


In [15]:
import networkx as nx

from collections import deque
from heapq import heappop, heappush


tests_part_I = [
    {
        "name": "Example 1",
        "grid": """
            #########
            #b.A.@.a#
            #########
        """,
        "expected": 8,
    },
    {
        "name": "Example 2",
        "grid": """
            ########################
            #f.D.E.e.C.b.A.@.a.B.c.#
            ######################.#
            #d.....................#
            ########################
        """,
        "expected": 86,
    },
    {
        "name": "Example 3",
        "grid": """
            ########################
            #...............b.C.D.f#
            #.######################
            #.....@.a.B.c.d.A.e.F.g#
            ########################
        """,
        "expected": 132,
    },
    {
        "name": "Example 4",
        "grid": """
            #################
            #i.G..c...e..H.p#
            ########.########
            #j.A..b...f..D.o#
            ########@########
            #k.E..a...g..B.n#
            ########.########
            #l.F..d...h..C.m#
            #################
        """,
        "expected": 136,
    },
    {
        "name": "Example 5",
        "grid": """
            ########################
            #@..............ac.GI.b#
            ###d#e#f################
            ###A#B#C################
            ###g#h#i################
            ########################
        """,
        "expected": 81,
    },
]


class Map:
    directions = ((-1, 0), (0, 1), (1, 0), (0, -1))

    def __init__(self, grid: str) -> None:
        self.grid = [list(line.strip()) for line in grid.strip().splitlines()]
        self.rows, self.cols = len(self.grid), len(self.grid[0])

        self.doors_and_keys()
        self.make_graph()

    def doors_and_keys(self) -> None:
        self.start = None
        self.keys, self.doors = {}, {}

        for r, c in product(range(self.rows), range(self.cols)):
            if self.grid[r][c].islower():
                self.keys[self.grid[r][c]] = r, c
            elif self.grid[r][c].isupper():
                self.doors[self.grid[r][c]] = r, c
            elif self.grid[r][c] == "@":
                self.start = r, c

    def make_graph(self) -> None:
        self.graph = {k: {} for k in self.keys.keys() | self.doors.keys()}
        self.graph["@"] = {}

        start_points = [("@", 0, *self.start)] + [
            (k, 0, *v) for k, v in (self.keys | self.doors).items()
        ]

        for p in start_points:
            queue = deque([p])
            seen = set()

            while queue:
                fr, d, r, c = queue.popleft()

                if (
                    not 0 <= r < self.rows
                    or not 0 <= c < self.cols
                    or self.grid[r][c] == "#"
                    or (r, c) in seen
                ):
                    continue

                seen.add((r, c))

                if self.grid[r][c] != fr and self.grid[r][c] != ".":
                    self.graph[fr][self.grid[r][c]] = d
                else:
                    queue.extend(
                        (fr, d + 1, r + dr, c + dc) for dr, dc in self.directions
                    )

    def shortest_path_all_keys(self) -> int:
        nr_keys = len(self.keys)
        seen = set()
        heap = [(0, "@", frozenset())]

        while heap:
            steps, current, keys = heappop(heap)

            if current.islower():
                keys |= {current}

            if len(keys) == nr_keys:
                return steps

            if (current, keys) in seen:
                continue

            seen.add((current, keys))

            for neigbor, delta_steps in self.graph[current].items():
                if not (neigbor.isupper() and neigbor.lower() not in keys):
                    heappush(
                        heap,
                        (steps + delta_steps, neigbor, keys),
                    )

    def plot_graph(self) -> None:
        G = nx.Graph()  # create an empty object

        G.add_nodes_from(self.graph.keys())
        G.add_edges_from(
            (fr, to) for fr, tos in self.graph.items() for to in tos.keys()
        )
        edge_labels = {
            (fr, to): w for fr, tos in self.graph.items() for to, w in tos.items()
        }
        pos = nx.bfs_layout(G, "@")
        # layers = {"a": ["@"], "b": list(self.keys.keys()), "c": list(self.doors.keys())}
        # pos = nx.multipartite_layout(G, subset_key=layers)
        nx.draw(G, pos, with_labels=True, node_color="MediumOrchid")
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="red")
        plt.show()
        plt.close()

    def __repr__(self) -> str:
        return "\n".join(
            [
                "\n".join("".join(f"{c}" for c in l) for l in self.grid),
                f"\n{self.keys=}, \n{self.doors=} \n{self.start=}",
                "",
                f"\n".join(
                    f"{k}: {', '.join(f'{k1}={v1}' for k1, v1 in v.items())}"
                    for k, v in self.graph.items()
                ),
            ]
        )


def tester(grid):
    return Map(grid).shortest_path_all_keys()


#
run_tests_params(tester, tests_part_I)

# plt.rcParams["figure.figsize"] = [12, 8]
# for test in tests:
#     Map(test["grid"]).plot_graph()


Test Example 1 passed, for tester.
Test Example 2 passed, for tester.
Test Example 3 passed, for tester.
Test Example 4 passed, for tester.
Test Example 5 passed, for tester.
Success


In [16]:
with open("../input/day18.txt") as f:
    puzzle = f.read()

print(f"Part I: {Map(puzzle).shortest_path_all_keys()}")  # 4566 too high

# m = Map(puzzle)
# m.plot_graph()

Part I: 4544


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4544</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You arrive at the vault only to <span title="To see the inspiration for this puzzle, look up 'Link to the Past Randomizer Multiworld'.">discover</span> that there is not one vault, but <em>four</em> - each with its own entrance.</p>
<p>On your map, find the area in the middle that looks like this:</p>
<pre><code>...
.@.
...
</code></pre>
<p>Update your map to instead use the correct data:</p>
<pre><code>@#@
###
@#@
</code></pre>
<p>This change will split your map into four separate sections, each with its own entrance:</p>
<pre><code>#######       #######
#a.#Cd#       #a.#Cd#
##...##       ##<em>@#@</em>##
##.@.##  --&gt;  ##<em>###</em>##
##...##       ##<em>@#@</em>##
#cB#Ab#       #cB#Ab#
#######       #######
</code></pre>
<p>Because some of the keys are for doors in other vaults, it would take much too long to collect all of the keys by yourself.  Instead, you deploy four remote-controlled robots. Each starts at one of the entrances (<code>@</code>).</p>
<p>Your goal is still to <em>collect all of the keys in the fewest steps</em>, but now, each robot has its own position and can move independently.  You can only remotely control a single robot at a time. Collecting a key instantly unlocks any corresponding doors, regardless of the vault in which the key or door is found.</p>
<p>For example, in the map above, the top-left robot first collects key <code>a</code>, unlocking door <code>A</code> in the bottom-right vault:</p>
<pre><code>#######
#@.#Cd#
##.#@##
#######
##@#@##
#cB#.b#
#######
</code></pre>
<p>Then, the bottom-right robot collects key <code>b</code>, unlocking door <code>B</code> in the bottom-left vault:</p>
<pre><code>#######
#@.#Cd#
##.#@##
#######
##@#.##
#c.#.@#
#######
</code></pre>
<p>Then, the bottom-left robot collects key <code>c</code>:</p>
<pre><code>#######
#@.#.d#
##.#@##
#######
##.#.##
#@.#.@#
#######
</code></pre>
<p>Finally, the top-right robot collects key <code>d</code>:</p>
<pre><code>#######
#@.#.@#
##.#.##
#######
##.#.##
#@.#.@#
#######
</code></pre>
<p>In this example, it only took <code><em>8</em></code> steps to collect all of the keys.</p>
<p>Sometimes, multiple robots might have keys available, or a robot might have to wait for multiple keys to be collected:</p>
<pre><code>###############
#d.ABC.#.....a#
######@#@######
###############
######@#@######
#b.....#.....c#
###############
</code></pre>
<p>First, the top-right, bottom-left, and bottom-right robots take turns collecting keys <code>a</code>, <code>b</code>, and <code>c</code>, a total of <code>6 + 6 + 6 = 18</code> steps. Then, the top-left robot can access key <code>d</code>, spending another <code>6</code> steps; collecting all of the keys here takes a minimum of <code><em>24</em></code> steps.</p>
<p>Here's a more complex example:</p>
<pre><code>#############
#DcBa.#.GhKl#
#.###@#@#I###
#e#d#####j#k#
###C#@#@###J#
#fEbA.#.FgHi#
#############
</code></pre>
<ul>
<li>Top-left robot collects key <code>a</code>.</li>
<li>Bottom-left robot collects key <code>b</code>.</li>
<li>Top-left robot collects key <code>c</code>.</li>
<li>Bottom-left robot collects key <code>d</code>.</li>
<li>Top-left robot collects key <code>e</code>.</li>
<li>Bottom-left robot collects key <code>f</code>.</li>
<li>Bottom-right robot collects key <code>g</code>.</li>
<li>Top-right robot collects key <code>h</code>.</li>
<li>Bottom-right robot collects key <code>i</code>.</li>
<li>Top-right robot collects key <code>j</code>.</li>
<li>Bottom-right robot collects key <code>k</code>.</li>
<li>Top-right robot collects key <code>l</code>.</li>
</ul>
<p>In the above example, the fewest steps to collect all of the keys is <code><em>32</em></code>.</p>
<p>Here's an example with more choices:</p>
<pre><code>#############
#g#f.D#..h#l#
#F###e#E###.#
#dCba@#@BcIJ#
#############
#nK.L@#@G...#
#M###N#H###.#
#o#m..#i#jk.#
#############
</code></pre>
<p>One solution with the fewest steps is:</p>
<ul>
<li>Top-left robot collects key <code>e</code>.</li>
<li>Top-right robot collects key <code>h</code>.</li>
<li>Bottom-right robot collects key <code>i</code>.</li>
<li>Top-left robot collects key <code>a</code>.</li>
<li>Top-left robot collects key <code>b</code>.</li>
<li>Top-right robot collects key <code>c</code>.</li>
<li>Top-left robot collects key <code>d</code>.</li>
<li>Top-left robot collects key <code>f</code>.</li>
<li>Top-left robot collects key <code>g</code>.</li>
<li>Bottom-right robot collects key <code>k</code>.</li>
<li>Bottom-right robot collects key <code>j</code>.</li>
<li>Top-right robot collects key <code>l</code>.</li>
<li>Bottom-left robot collects key <code>n</code>.</li>
<li>Bottom-left robot collects key <code>m</code>.</li>
<li>Bottom-left robot collects key <code>o</code>.</li>
</ul>
<p>This example requires at least <code><em>72</em></code> steps to collect all keys.</p>
<p>After updating your map and using the remote-controlled robots, <em>what is the fewest steps necessary to collect all of the keys?</em></p>
</article>

</main>


In [17]:
from copy import copy, deepcopy
import networkx as nx

from collections import deque
from heapq import heappop, heappush


tests_part_II = [
    {
        "name": "Example 1",
        "grid": """
            #######
            #a.#Cd#
            ##...##
            ##.@.##
            ##...##
            #cB#Ab#
            #######
        """,
        "expected": 8,
    },
    {
        "name": "Example 2",
        "grid": """
            ###############
            #d.ABC.#.....a#
            ######...######
            ######.@.######
            ######...######
            #b.....#.....c#
            ###############
        """,
        "expected": 24,
    },
    {
        "name": "Example 3",
        "grid": """
            #############
            #DcBa.#.GhKl#
            #.###...#I###
            #e#d#.@.#j#k#
            ###C#...###J#
            #fEbA.#.FgHi#
            #############
        """,
        "expected": 32,
    },
    {
        "name": "Example 4",
        "grid": """
            #############
            #g#f.D#..h#l#
            #F###e#E###.#
            #dCba...BcIJ#
            #####.@.#####
            #nK.L...G...#
            #M###N#H###.#
            #o#m..#i#jk.#
            #############
        """,
        "expected": 72,
    },
]


class MapII:
    directions = ((-1, 0), (0, 1), (1, 0), (0, -1))

    def __init__(self, grid: str) -> None:
        self.grid = [list(line.strip()) for line in grid.strip().splitlines()]
        self.rows, self.cols = len(self.grid), len(self.grid[0])

        self.doors_and_keys()
        self.make_graph()

    def doors_and_keys(self) -> None:
        self.start = {}
        self.keys, self.doors, self.start = {}, {}, {}

        r_start, c_start = -1, -1

        for r, c in product(range(self.rows), range(self.cols)):
            if self.grid[r][c].islower():
                self.keys[self.grid[r][c]] = r, c
            elif self.grid[r][c].isupper():
                self.doors[self.grid[r][c]] = r, c
            elif self.grid[r][c] == "@":
                r_start, c_start = r, c

        self.grid[r_start - 1][c_start - 1] = "@1"
        self.grid[r_start - 1][c_start] = "#"
        self.grid[r_start - 1][c_start + 1] = "@2"
        self.grid[r_start][c_start] = "#"
        self.grid[r_start + 1][c_start - 1] = "@3"
        self.grid[r_start + 1][c_start] = "#"
        self.grid[r_start + 1][c_start + 1] = "@4"
        self.grid[r_start][c_start - 1] = "#"
        self.grid[r_start][c_start + 1] = "#"

        self.start = {
            "@1": (r_start - 1, c_start - 1),
            "@2": (r_start - 1, c_start + 1),
            "@3": (r_start + 1, c_start - 1),
            "@4": (r_start + 1, c_start + 1),
        }

    def make_graph(self) -> None:
        self.graph = {k: {} for k in self.start | self.keys.keys() | self.doors.keys()}

        start_points = [
            (k, 0, *v) for k, v in (self.start | self.keys | self.doors).items()
        ]

        for p in start_points:
            queue = deque([p])
            seen = set()

            while queue:
                fr, d, r, c = queue.popleft()

                if (
                    not 0 <= r < self.rows
                    or not 0 <= c < self.cols
                    or self.grid[r][c] == "#"
                    or (r, c) in seen
                ):
                    continue

                seen.add((r, c))

                if self.grid[r][c] != fr and self.grid[r][c] != ".":
                    self.graph[fr][self.grid[r][c]] = d
                else:
                    queue.extend(
                        (fr, d + 1, r + dr, c + dc) for dr, dc in self.directions
                    )

    def shortest_path_all_keys(self) -> int:
        nr_keys = len(self.keys)
        seen = set()
        heap = [(0, 0, tuple(self.start.keys()), frozenset())]

        while heap:
            steps, active, currents, keys = heappop(heap)

            if currents[active].islower():
                keys |= {currents[active]}

            if len(keys) == nr_keys:
                return steps

            if (currents, keys) in seen:
                continue

            seen.add((currents, keys))

            for active1, current in enumerate(currents):
                for neigbor, delta_steps in self.graph[current].items():
                    if neigbor.isupper() and neigbor.lower() not in keys:
                        continue

                    heappush(
                        heap,
                        (
                            steps + delta_steps,
                            active1,
                            currents[:active1] + (neigbor,) + currents[active1 + 1 :],
                            keys,
                        ),
                    )

    def plot_graph(self) -> None:
        G = nx.Graph()  # create an empty object

        G.add_nodes_from(self.graph.keys())
        G.add_edges_from(
            (fr, to) for fr, tos in self.graph.items() for to in tos.keys()
        )
        edge_labels = {
            (fr, to): w for fr, tos in self.graph.items() for to, w in tos.items()
        }

        pos = nx.planar_layout(G)

        nx.draw(G, pos, with_labels=True, node_color="MediumOrchid")
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="red")
        plt.show()
        plt.close()
        print(f"{sum(1 for _ in nx.connected_components(G))=}")

    def __repr__(self) -> str:
        return "\n".join(
            [
                "\n".join("".join(f"{c}" for c in l) for l in self.grid),
                f"\n{self.keys=}, \n{self.doors=} \n{self.start=}",
                "",
                # f"\n".join(
                #     f"{k}: {', '.join(f'{k1}={v1}' for k1, v1 in v.items())}"
                #     for k, v in self.graph.items()
                # ),
            ]
        )


def tester(grid):
    return MapII(grid).shortest_path_all_keys()


#
run_tests_params(tester, tests_part_II)

# plt.rcParams["figure.figsize"] = [14, 10]
# for test in tests_part_II:
#     MapII(test["grid"]).plot_graph()


Test Example 1 passed, for tester.
Test Example 2 passed, for tester.
Test Example 3 passed, for tester.
Test Example 4 passed, for tester.
Success


In [19]:
print(f"Part II: {MapII(puzzle).shortest_path_all_keys()}")

Part II: 1692


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1692</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
